# Przetwarzanie równoległe

W Julii mamy cztery typy obliczeń równoległych:

1. Asynchroniczne zadania (`tasks`) - które mogą być zawieszane, przełączane, oczekiwać zakończenia przez inne zadania itp. Ściśle rzecz biorąc nie są to obliczenia równoległe, ale są takie kiedy zadania są wykonywane przez różne wątki
2. Obliczenia wielowątkowe - zadania uruchamiane na wielu wątkach, na wielu procesorach, ale na tej samej maszynie, dzielące wspólną pamięć RAM
3. Obliczenia rozproszone - obliczenia wykonywane na oddzielnych przestrzeniach pamięci na wielu procesorach lub komputerach
4. Obliczenia na GPU

Na zajęciach zajmiemy się przypadkami 1, 2 i 3. Obliczenia na GPU wymagają pewnych przygotowań oraz przede wszystkim posiadania odpowiedniego GPU (którego nie możemy zapewnić) zaintersowanych odsyłam [tu](https://juliagpu.org/).

## 1. Zadania (tasks)

Kawałek kodu (zwykle funkcja) może zostać zadaniem (`task`). W odróżnieniu od zwykłych funkcji mamy swobodę przełączania się między zadaniami w dowolnej kolejności. Typowym zastosowaniem jest np. komunikacja z zewnętrzem, czekanie na transfer danych itp. podczas którego możemy wykonywać inne czynności i przerwać je gdy zajdzie potrzeba.

Zadanie tworzymy za pomocą makra `@task`, następujące wyrażenie (funkcja, itp.) zostanie zadaniem, ale nie zostanie uruchomione.

In [ ]:
y = rand(1000)
t = @task begin; sleep(2); println("t: ", length(filter(x -> x > 0.5, y))); end

Zadanie uruchamiamy poleceniem `schedule`, zostanie ono uruchomione i program przejdzie do następnych poleceń. Jeżeli chemy od razu stworzyć i uruchomić zadanie możemy użyć polecenia `@async`.

In [ ]:
schedule(t)
println("main: ", length(filter(x -> x > 0.5, y)))
@async begin; sleep(2); println("t2: done"); end

Jeżeli chcemy w którymś momencie zaczekać na zakończenie zadania, możemy użyć polecenia `wait(t)`.

### Przykład

Wyobraźmy sobie, że checmy sprawdzić jakie komputery odpowiadają na wywołanie w podsieci x.x.x.x/24, czyli np. 10.13.2.1-254. Użyjemy systemowego polecenia `ping`, których będziemy próbować wywołać komputer tylko jeden raz i czekać na odpowiedź co najwyżej 5 sekund. Polecenia zewnętrzne w Julii (podobnie jak w shellu) otaczane są odwrotnymi pojedynczymi cudczysłowami \`. Polecenie można uruchomić funkcją `run`, albo jeżeli chcemy poznać odpowiedź `read`.

In [ ]:
read(`ping -c 1 -w 5 192.168.1.1`, String)

W przypadku gdy dany adres nie odpowiada otrzymamy błąd, bo proces zwraca wynik różny niż zero.

In [ ]:
read(`ping -c 1 -w 5 192.168.1.2`, String)

Skanowanie sieci kolejnymi wywołaniami może trwać długo - jeżeli na każdą odpowiedź czekamy maksymalnie 5 sekund i dopiero wtedy wysyłamy następny ping, to łącznie może to trwać ponad 20 minut (254 * 5 sekund). Możemy jednak uruchomić równolegle wiele procesów. Na koniec poczekamy aż wszystkie zadania skończą działanie i wyświetlimy listę adresów IP, które odpowiedziały.

In [ ]:
function ping(ip)
    try
        s = read(`ping -c 1 -w 5 $ip`, String)
        return s
    catch err
        return ""
    end
end

In [ ]:
N = 254
ip_tab = Array{String, 1}(undef, N)
tasks = Array{Task, 1}()
for i in 1:N
    push!(tasks, @async ip_tab[i] = ping("192.168.1.$i"))
end

for i in 1:N
    wait(tasks[i])
    if length(ip_tab[i]) > 0
        println("192.168.1.$i")
    end
end

Podobny efekt możemy osiągnąć używając funkcji `asyncmap`, która jest odpowiednikiem `map`, ale zamiast uruchamiać funckje dla każdego elementu kolekcji, uruchomi zadania

In [ ]:
asyncmap(x->ping(x), 1:N);

## Kanały (Channel)

Kanał to kolejka FIFO o zadanej długości (lub "nieskończona"), której mogą używać różne zadania do komunikacji i wymiany danych.
* Channel{T}(n) - typ T, długość n; domyślnie Any, 0 - kolejka bez bufora, długość Inf oznacza typemax(Int)
* `put!` dodaje element do kolejki
* `take!` zabiera element z kolejki
* `fetch` czyta element z kolejki (bez zabierania)\
* jeżeli kanał jest pusty funkcja `take!` czeka aż pojawią się dane
* jeżeli kanał jest pełny funkcja `put!` czeka aż pojawi się miejsce
* `isready` sprawdza obecność jakiegokolwiek elementu w kolejce
* `wait` czeka na pojawienie się elementu

In [ ]:
using GRUtils
using StatsBase

function worker(id, data, stop)
    while true
        if isready(stop)
            break
        end
        sleep(rand() * 0.1)
        put!(data, id)
    end
    println("W-", id, " done")
end

sigstop = Channel{Bool}(1)
times = Channel{Int64}(Inf)
tasks = Array{Task, 1}()
N = 10

println("Start")

for i in 1:N
    push!(tasks, @async worker(i, times, sigstop))
end

sleep(5)
println("Stop send")
put!(sigstop, true)

for i in 1:N
    wait(tasks[i])
end

h = fit(Histogram, times.data, 1:1:N+1)
barplot(h.weights)

## Wątki

Przedstawione zadania nie są równoległe, bo używają pojedynczego wątku i wspólnej pamięci. Dlatego ich użycie nie przyspiesza np. obliczeń numerycznych, i są stosowane przede wszystkim do zadań I/O, aby nie blokować wykonywania innych części programu.

Prawdziwą równoległość umożliwiają wątki (threads). Domyślnie Julia startuje tylko z jednym wątkiem

In [ ]:
Threads.nthreads()

Ale uruchomiona z flagą -t/--threads może mieć ich więcej
```
$ julia --threads 4
```

W przypadku notatnika Jupyter trzeba zdefiniować nowy rdzeń np. z 4 wątkami

```
julia> using IJulia
julia> IJulia.installkernel("Julia 4 Threads", env=Dict("JULIA_NUM_THREADS" => "4",))
```

Używanie wątków oprócz oczywistych korzyści sprawia także dodatkowe zobowiązania takie jak zapewnienie, że nie występuje wyścig do danych (data-race) potencjalne zakleszczenie (deadlock), zagłodzenie (starvation), oraz czytanie danych, które mogą być manipulowane przez wiele wątków jest bezpieczne.

Aby zapewnić brak negatywnych zjawisk istnieje kilka mechanizów - blokady oraz operacje atomistyczne.

Do ilustracji wyścigu użyjemy jednego z najprostszych mechanizów wielowąkowych - makra `@threads`, które automatycznie zrównolegla pętle `for`. W przykładzie używamy tablicy jednoelementowej ze względu na sposób przekazywania przez wskaźnik.

In [ ]:
# Wersja jednowątkowa
s = zeros(Int64, 1)
for i in 1:1_000_000
    s[1] += 1
end
println(s)

W przypadku prostej pętli nie mamy żadnego problemu z oczywistem wynikiem dodawania jedynki milion razy

In [ ]:
# Wielowątkowa
s = zeros(Int64, 1)
Threads.@threads for i in 1:1_000_000
    s[1] += 1
end
println(s)

Naiwna wersja wielowątkowa po prostu dodaje makro przed pętlą for, ale nie dba o wystąpienie wyścigu: wątki wykonują operację dodawania w 3 krokach (pobierz, dodaj, wpisz)

| Wątek i operacja | Wartość |
-------------------|---------|
| 1: pobiera 0     |      0    |
| 2: pobiera 0     |      0    |
| 1: dodaje 0+1    |      0    |
| 2: dodaje 0+1    |      0    |
| 1: wpisuje 1     |      1    |
| 2: wpisuje 1     |      1    |

In [ ]:
# Atomistyczne
s = Threads.Atomic{Int}(0)
Threads.@threads for i in 1:1_000_000
    Threads.atomic_add!(s, 1)
end
println(s.value)

Operacja atomistyczna (jednostkowa) zapewnia, że cała czynność musi być wykonana w jednym kroku (choć na poziomie instrukcji procesora nadal musi być ich kilka). Dzięki temu dodawanie atomistyczne (atomic_add!) daje dobry wynik.

Ten sam efekt można osiągnąć stosując blokadę (lock), która zapewnia, że tylko jeden wątek wykonuje krytyczną operację jednocześnie. Oczywiście w tym prostym przypadku operacja atomistyczna jest prostsza w implementacji, ale mechanizm blokad jest znacznie bardziej elastyczny i w bardziej skomplikowanym przypadku może być niezbędny.

Podstawowy schemat użycia blokady to:
```
    lock(lk)
    try
        # operacja
    finally
        unlock(lk)
    end
```

In [ ]:
function increase(s, x, lk)
    lock(lk)
    try
        s[1] += x
    finally
        unlock(lk)
    end
end

rl = ReentrantLock()
s = zeros(Int64, 1)
Threads.@threads for i in 1:1_000_000
    increase(s, 1, rl)
end
println(s)

Makro `@threads` działa tylko na pętle `for`. W ogólnym przypadku uruchamianie wątków jest bardzo podobne do uruchamiania zadań. Makro `@spawn` jest odpowiednikiem `@async`. Pozostałe mechanizmy (wait, channel, itp.) można używać jak w programowaniu asynchronicznym. Różnicą jest oczywiście uruchamianie wątków na osobnych wątkach CPU i lepsze wykorzystanie mocy obliczeniowych komputera.

In [ ]:
using GRUtils
using StatsBase

function worker(id, data, stop)
    while true
        if isready(stop)
            break
        end
        for i in 1:1_000_000
            rand()
        end
        put!(data, id)
    end
    println("W-", id, " done")
end

sigstop = Channel{Bool}(1)
times = Channel{Int64}(Inf)
tasks = Array{Task, 1}()
N = 4

println("Start")

for i in 1:N
    push!(tasks, Threads.@spawn worker(i, times, sigstop))
end

sleep(5)
println("Stop send")
put!(sigstop, true)

for i in 1:N
    wait(tasks[i])
end

h = fit(Histogram, times.data, 1:1:N+1)
barplot(h.weights)

## Przetwarzanie wieloprocesorowe i rozproszone

Pełne wykorzystanie zasobów współczesnych komputerów posiadających wiele procesorów, czy klastrów komputerów lub wręcz wykorzystanie rozproszonych fizycznie komputerów wymaga jeszcze innego podejścia, w którym nie tylko CPU, ale i pamięć RAM jest rozdzielona pomiędzy procesory. Rodzi to dodatkowe problemy w programowaniu i zasadniczo różni się ono od tego co widzieliśmy dotychczas.

Aby uruchomić Julię korzystając z wielu procesów podajemy parametr `-p n`, co automatycznie ładuje bibliotekę `Distributed` i `n` dodatkowych procesów. Oczywiście sensownie jest ustalić tę wielkość na nie więcej niż liczba fizycznych rdzeni procesora. Każdy proces ma swój indetyfikator zaczynający się od 1 (jeżeli uruchomiona jest interaktywna sesja, to zawsze ma numer 1). Numer można sprawdzić przez `myid()`, a listę wszystkich "robotników" (workers - procesy poza głównym) przez `workers()`. Dodawać procesy można także ładując bibliotekę i używając funkcji `addprocs`.

In [ ]:
using Distributed

println("myid: ", myid())
println(workers())
println(nprocs())
println(nworkers())

addprocs(2)
println(workers())
println(nprocs())
println(nworkers())

Podstawową niskopoziomową strukturą jest uruchomienie wyrażenia (funkcji, itp.) na procesie `n` poprzez `remotecall`. Zwraca ono obiekt `Future`, który jest zastępczym obiektem dla wyniku obliczeń, które mają nieznany stan i czas zakończenia.

```
remotecall(funkcja, worker_id, argumenty...)
```

Funkcja `fetch` czeka na zakończenie procesu i czyta zwracaną wartość.

In [ ]:
r = remotecall(rand, 2, Int64, 10)

In [ ]:
fetch(r)

Makro `@spawnat p wyrażenie`  uruchamia wyrażenie na procesie `p`, zamiast numeru można użyć `:any` i wtedy proces zostanie wybrany automatycznie.

In [ ]:
r = @spawnat :any rand(10)

In [ ]:
fetch(r)

Kod który uruchamiamy musi być dostępny dla wszystkich procesów. Jeżeli zdefinujemy np. jakąś funkcję w sesji interaktywnej (proces 1), to inne procesy nie będą jej znały

In [ ]:
function f()
    rand(10)
end

r = @spawnat 2 f()

In [ ]:
fetch(r)

Makro `@everywhere` pozwala na załadowanie wyrażenia na wszystkich procesach

In [ ]:
@everywhere function f()
                rand(10)
end

In [ ]:
r = @spawnat 2 f()
fetch(r)

Ponieważ każdy proces ma swoją pamięć RAM, czasami konieczne jest przesyłanie danych pomiędzy procesami. Jest to zwykle wąskie gardło obliczeń równoległych i powinno być minimalizowane. Najlepiej jest tak zorganizować algorytm czy kod, aby zwracał on wynik obliczeń i minimalizował przesyłanie danych.

Odpowiednikiem kanałów jest `RemoteChannel`. Inne struktury też mają swoje odpowiedniki np. `SharedArrays`. Użyjemy tych dwóch struktur w przykładzie poniżej.

## Przykład 

Spróbujemy zbudować następujący wieloprocesorowy program, w którym dane będą generowane dane przez niezależne procesy, a trzeci proces będzie zajęty rysowaniem wykresu (histogramu). Jeden generator będzie losował szum, drugi rozkład normalny. Obydwa będą wpisywały wyniki do współdzielonej tablicy. Użyjemy podobnego mechanizmu zatrzymywania co wcześniej, poprzez wysłanie sygnału do wszystkich procesów.

In [ ]:
using Distributed
using SharedArrays
using GRUtils

addprocs(2)

@everywhere using Distributions



Generator szumu będzie produkował dane z rozkładu płaskiego w całym zakresie histogramu. Zwróci na koniec liczbę punktów, którą wpisał do histogramu (będziemy mogli sprawdzić czy nie straciliśmy danych).

In [ ]:
@everywhere function generator_noise!(sigstop, histogram)
    println("Starting noise generator ", myid())
    nmax = length(histogram)
    n = 0
    while true
        if isready(sigstop)
            break
        end
        i = round(Int64, rand() * nmax, RoundUp)
        if 0 < i <= nmax
            histogram[i] += 1
            n += 1
        end
    end
    println("Noise generator done")
    n
end


Drugi generator będzie produkował dane z rozkładu normalnego

In [ ]:
@everywhere function generator_peak!(sigstop, histogram)
    println("Starting peak generator ", myid())
    nmax = length(histogram)
    normal = Normal(nmax / 2, 100.0)
    n = 0
    while true
        if isready(sigstop)
            break
        end
        i = round(Int64, rand(normal), RoundUp)
        if 0 < i <= nmax
            histogram[i] += 1
            n += 1
        end
    end
    println("Peak generator done")
    n
end

Potrzebujemy także funkcji rysującej bieżący histogram. Oprócz danych i sygnałów stopu zdefinujemy co ile sekund wykres ma się odświeżać.

In [ ]:
function plotter(sigstop, histogram, fig, update)
    println("Starting plotter ", myid())

    gcf(fig)
    while true
        if isready(sigstop)
            break
        end
        stairs(histogram)
        xlim(0, 16384)
        xticks(1000, 5)
        ylim(0, maximum(histogram) + 5)
        display(gcf())
        sleep(update)
    end
    println("Plotter done")
end


Zdefinujemy teraz sygnał stopujący (RemoteChannel), histogram (SharedArray) i wykres. Funckja rysująca będzie zadaniem, bo ma odświeżać wykres tylko co jakiś czas. Zmienna trun zdefinuje czas działania w sekundach, po jakim wysłany zostanie sygnał stopujący.

In [ ]:
trun = 10

sigstop = RemoteChannel(()->Channel{Bool}(1))
histogram = SharedArray{Int64, 1}(16384)

#fig = Figure((800, 600), "pix")
#display(gcf())

#p = @task plotter(sigstop, histogram, fig, 0.1)
#schedule(p)

gn = @spawnat :any generator_noise!(sigstop, histogram)
gp = @spawnat :any generator_peak!(sigstop, histogram)

sleep(trun)
put!(sigstop, true)

sn = fetch(gn)
sp = fetch(gn)
println(sum(histogram), " events in histogram")
println(sn + sp, " events generated")


In [ ]:
plot(histogram)